In [36]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pymongo import MongoClient
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import word_tokenize
import gensim
from pprint import pprint
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import spacy
from nltk.corpus import stopwords
import pickle
from nltk.stem import PorterStemmer
from random import sample 

In [30]:
client = MongoClient('localhost',27017)

In [31]:
db = client['covid']
studies = db.studies 

In [4]:
doc_path = '/Users/jeloretizo/Library/Mobile Documents/com~apple~CloudDocs/CORD-19-research-challenge'

In [5]:
metadata = pd.read_csv(doc_path + '/metadata.csv')

In [115]:
metadata.loc[metadata['sha'] == 'ff766484e50701bbb36ed45d3a4464d817a471ff']['abstract'].iloc[0]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
40896,nl37p9vp,ff766484e50701bbb36ed45d3a4464d817a471ff,Elsevier,Middle East Respiratory Syndrome Interpreted: ...,10.1016/j.amjmed.2016.04.030,PMC7124269,27215907.0,els-covid,NaN,2016-09-30,"Lau, Susanna K.P.; Chan, Jasper F.W.; Hung, Iv...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.amjmed.2016.04.030


In [70]:
comm_pmc_json = doc_path + '/comm_use_subset/comm_use_subset/pmc_json/'
comm_pdf_json = doc_path + '/comm_use_subset/comm_use_subset/pdf_json/'
custom_pmc_json = doc_path + '/custom_license/custom_license/pmc_json/'
custom_pdf_json = doc_path + '/custom_license/custom_license/pdf_json/'

In [71]:
comm_pmc_json_files = [f for f in listdir(comm_pmc_json) if isfile(join(comm_pmc_json, f))]
comm_pdf_json_files = [f for f in listdir(comm_pdf_json) if isfile(join(comm_pdf_json, f))]
custom_pmc_json_files = [f for f in listdir(custom_pmc_json) if isfile(join(custom_pmc_json, f))]
custom_pdf_json_files = [f for f in listdir(custom_pdf_json) if isfile(join(custom_pdf_json, f))]

In [8]:
len(comm_pdf_json_files)

9524

In [72]:
len(comm_pmc_json_files)

9148

In [74]:
# comm_pmc_json_files

In [10]:
len(custom_pdf_json_files)

26505

In [11]:
len(custom_pmc_json_files)

7797

In [ ]:
def find_abstract(research_id):
    ''' Helper function to aggregate abstract and metadata '''
    if research_id is not None:
        if 'PMC' in research_id:
            try:
                abstract = metadata.loc[metadata['pmcid'] == research_id]['abstract'].iloc[0]
            except:
                abstract = None
        else:
            try: 
                abstract = metadata.loc[metadata['sha'] == research_id]['abstract'].iloc[0]
            except:
                abstract = None
    else:
        abstract = None
    return abstract

In [82]:
def load_files(files_list, files_path):
    for orig_file in tqdm(files_list):
        with open((files_path + orig_file), 'r') as file:
            try:
                file = json.load(file)
    #             print(custom_pdf_json_file)
            except:
                print('Warning! File not read:', orig_file)
            res = {}
            try:
                res['paper_id'] = file['paper_id'] 
            except:
                res['paper_id'] = None

            try:
                res['abstract'] = file['abstract'][0]['text']
            except:
                res['abstract'] = None

            try: 
                res['title'] = file['metadata']['title']
            except:
                res['title'] = None

            try:
                res['authors'] = file['metadata']['authors']
            except:
                res['authors'] = None

            try:
                res['other_metadata'] = file['metadata']
            except:
                res['other_metadata'] = None

            try:
                res['other_abstract'] = file['abstract']
            except:
                res['other_abstract'] = None

    #         data = {}
            full_text = ''
            try:
    #             for i in file['body_text']:
    #                 section = i['section'].lower()
    #                 if '.' in section:
    #                     section = section.replace('.', ' ')
    #                 if section in list(data.keys()): 
    #                     new_text = data[section] + "\n" + i['text']
    #                     data[section] = new_text
    #     #                 new_text
    #     #                 data[i['section']].append(i['text'])
    #             #         print('a')
    #                 else:
    #                     data[section] = i['text']
                for i in file['body_text']:
                    full_text += i['text']

            except:
                full_text = None

            try:
                res['body'] = full_text
            except:
                res['body'] = None
            try:
                studies.insert_one(res, bypass_document_validation=True)
            except:
                print('Warning! File not inserted:', orig_file)

In [83]:
load_files(custom_pdf_json_files, custom_pdf_json)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Warning! File not read: ffc89f2bd7f513dae147b4bd0043164db16e8dc0.json
Warning! File not read: ff966b5218e3c9433966d622a806710b820a60d6.json
Warning! File not read: ff9398e9978220ab54cd85a98920d90b05c190d6.json
Warning! File not read: ffb877d67fd30e14677580c292ca9caf64659987.json
Warning! File not read: ff6c4e420ae293c8130a3a71a1ab9e466ccf0b40.json
Warning! File not read: ff765aa3a876d263d37993712a503bc2a9f5ef0c.json
Warning! File not read: ff740fd015f63477d7b236d0ffee2b7639b6dcb0.json
Warning! File not read: ffc93dea5a5f4e6e65ad155e87aafa98aa8c54cf.json
Warning! File not read: ff80b3d9f398a7e1b51c1c2dc6fdbc97969d24f0.json
Warning! File not read: ffd3606ed51d3129db5863b2246ec587e07323c8.json
Warning! File not read: ffb77680c0f79491003331d1e3d9dde3c4fcdfd9.json
Warning! File not read: ffab348b231771ee27738a4426a87a6a04f3994f.json
Warning! File not read: ff7d49ac4008f60ef9c5a437e0d504dcefd1246f.json
Warning! File not read: ffd73c393641618860486e897197311f630bf67f.json
Warning! File not re

In [84]:
load_files(custom_pmc_json_files, custom_pmc_json)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Warning! File not read: PMC1383626.xml.json
Warning! File not read: PMC1850917.xml.json
Warning! File not read: PMC1635791.xml.json
Warning! File not read: PMC1160239.xml.json
Warning! File not read: PMC1518840.xml.json
Warning! File not read: PMC1182165.xml.json
Warning! File not read: PMC2259160.xml.json
Warning! File not read: PMC2206501.xml.json
Warning! File not read: PMC1592674.xml.json
Warning! File not read: PMC1566363.xml.json
Warning! File not read: PMC1769406.xml.json
Warning! File not read: PMC2435248.xml.json
Warning! File not read: PMC1474450.xml.json
Warning! File not read: PMC1278949.xml.json
Warning! File not read: PMC1074749.xml.json
Warning! File not read: PMC2589215.xml.json
Warning! File not read: PMC1821001.xml.json
Warning! File not read: PMC2516372.xml.json
Warning! File not read: PMC156578.xml.json
Warning! File not read: PMC1519403.xml.json
Warning! File not read: PMC1602263.xml.json
Warning! File not read: PMC2443186.xml.json
Warning! File not read: PMC182099

In [84]:
for i in file.keys():
    print(i)

paper_id
metadata
abstract
body_text
bib_entries
ref_entries
back_matter


In [85]:
file['paper_id']

'86a998617c077f4fe2ab26214995a3548fbc0fc5'

In [97]:
full_data = pd.DataFrame(list(studies.find(
    {}, {'_id': 0, 'authors': 0, 'other_metadata': 0, 'other_abstract': 0})))
full_data.drop('abstract', axis=1).to_csv('./all_research.csv', index=False)

In [63]:
full_data = pd.read_csv('./all_research.csv')

In [64]:
full_data = full_data.drop_duplicates(['title', 'new_abstract'], keep='first')

In [65]:
full_data = full_data.dropna()

In [66]:
full_data.to_csv('./new_research_base.csv', index=False)

In [47]:
subset = full_data.iloc[sample(range(full_data.shape[0]), 5000),]

In [48]:
subset['lower_body'] = subset['body'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
subset['lower_body_tokens'] = subset['lower_body'].apply(lambda x: word_tokenize(str(x)))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
subset['lower_body_tokens'] = subset['lower_body_tokens'].apply(lambda x: [w for w in x if w.isalpha()])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
stop_words = stopwords.words('english')
def remove_stopwords(word_tokens):
    return [w for w in word_tokens if not w in stop_words] 

In [52]:
# ps = PorterStemmer()
# def stem_words(tokens):
#     return [ps.stem(word) for word in tokens]

In [53]:
subset['lower_body_tokens'] = subset['lower_body_tokens'].apply(lambda x: remove_stopwords(x))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [54]:
# subset['lower_body_tokens'] = subset['lower_body_tokens'].apply(lambda x: stem_words(x))

In [55]:
all_body = list(subset['lower_body_tokens'])

In [56]:
model = gensim.models.Word2Vec(all_body, size=100, window=5, min_count=1)

In [57]:
word_vectors = model.wv

In [58]:
filename = 'finalized_model.pickle'
pickle.dump(model, open(filename, 'wb'))

In [59]:
model.most_similar(positive='coronavirus')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cov', 0.8557634353637695),
 ('coronaviruses', 0.8078387379646301),
 ('covs', 0.7044326066970825),
 ('astrovirus', 0.6497707962989807),
 ('hcov', 0.6493496894836426),
 ('picornavirus', 0.640275239944458),
 ('torovirus', 0.637744665145874),
 ('coronaviral', 0.6262747049331665),
 ('toroviruses', 0.6178733110427856),
 ('betacoronavirus', 0.6153151988983154)]

In [60]:
model.most_similar(positive='transmission')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('spread', 0.7335076332092285),
 ('transmissions', 0.720128059387207),
 ('spillover', 0.7012777924537659),
 ('dispersal', 0.696872353553772),
 ('contact', 0.6321157217025757),
 ('transmitted', 0.6225852966308594),
 ('zoonotic', 0.607498288154602),
 ('importation', 0.5951361656188965),
 ('outbreaks', 0.5944525003433228),
 ('spreading', 0.5937550067901611)]

In [68]:
search_string = 'What do we know about the transmission of the virus?'

In [81]:
new_search = ''
for i in r.get_ranked_phrases():
    new_search += ' ' + i

In [73]:
distances = [(i[1]['paper_id'], 
              model.wmdistance(search_string, 
                               i[1]['new_abstract'])) for i in full_data.iterrows()]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [67]:
# class research_model:
#     def search(search_string):
        

In [74]:
sorted(distances,key=lambda x: x[1])[:20]

[('PMC7097289', 2.6919014710587437),
 ('PMC7097751', 2.841968344435385),
 ('96bb7e80b35ce7f459c800fd26664447340faa53', 2.916129605593116),
 ('PMC7122602', 2.916129605593116),
 ('9bf1241b4b35efdf7be808ce9bc8d44939f1e21e', 3.0132828913403213),
 ('PMC1594563', 3.0132828913403213),
 ('7769931efb23130419643b2f2ce7bbca526c6c27', 3.066949052799813),
 ('ce74ed47b12de8d1c70b2cac843b6953c57bd5de', 3.1687591548836247),
 ('f316c6bd135b132aa6b9788723f1b677c7c01ab8', 3.2294702772364032),
 ('385c3a90b9d504dff3bc3769059cf82b85d423d9', 3.2441124637213),
 ('a4c0d3619ba273adce8a9d0df51eca6adaaf2f23', 3.2862045766669143),
 ('798546b8ea75ea9293e96ba46f4a1e390ea60b34', 3.2975291510525553),
 ('82b26efe64a82ae2cf599fa9cb90ced4ebfef2ce', 3.3244653373319286),
 ('b69fe5e231d41629e4620b18e90999022c72474d', 3.328294613745305),
 ('PMC7094977', 3.3318366483454676),
 ('PMC7122937', 3.3401454534995416),
 ('545220ce25bc633702c86f00f0c301efbc9bfb2a', 3.345211162946646),
 ('cb0831902de1f8a19992ff86bcc7e15fa2d7d687', 3.34

In [80]:
full_data.loc[full_data['paper_id'] == '96bb7e80b35ce7f459c800fd26664447340faa53']['title'].iloc[0]

'24 Bioinformatics'

In [82]:
full_data.loc[full_data['paper_id'] == '96bb7e80b35ce7f459c800fd26664447340faa53']['new_abstract'].iloc[0]

'After reading this chapter, you should know the answers to these questions:'

In [91]:
full_data.loc[full_data['paper_id'] == 'a6f478369adbfd8be967251a9bb0dd17fa436c45']['body']

9394    www.lungca.org 4 5 6 7 8 9 10 11 12 13 14 15 1...
Name: body, dtype: object

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')

In [7]:
doc_term_matrix = count_vect.fit_transform(subset['body'].values.astype('U'))

In [8]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.partial_fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [9]:
doc_term_matrix

<5000x80491 sparse matrix of type '<class 'numpy.int64'>'
	with 4602534 stored elements in Compressed Sparse Row format>

array([ 6661, 16776,  6161, ..., 31054, 11235,  2840])

In [10]:
first_topic =LDA.components_[0]

In [11]:
for i in first_topic.argsort()[-20:]:
    print(count_vect.get_feature_names()[i])

epidemic
human
countries
control
research
infectious
surveillance
diseases
case
outbreak
influenza
information
population
public
risk
transmission
model
cases
disease
health


In [12]:
count_vect.get_feature_names()[i]

'health'

In [13]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['influenza', 'information', 'population', 'public', 'risk', 'transmission', 'model', 'cases', 'disease', 'health']


Top 10 words for topic #1:
['infections', 'positive', 'viral', 'children', 'virus', 'samples', 'clinical', 'influenza', 'respiratory', 'patients']


Top 10 words for topic #2:
['genome', 'virus', 'viral', 'viruses', 'gene', 'sequence', 'sequences', 'genes', 'et', 'al']


Top 10 words for topic #3:
['activity', 'rna', 'figure', 'proteins', 'virus', 'viral', 'expression', 'protein', 'cell', 'cells']


Top 10 words for topic #4:
['antibodies', 'viral', 'protein', 'antibody', 'ml', 'infected', 'cell', 'mice', 'virus', 'cells']




In [14]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(5000, 5)

In [15]:
subset['Topic'] = topic_values.argmax(axis=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
subset['Topic'].unique()

array([4, 3, 2, 1, 0])

In [17]:
subset.head()

,paper_id,title,body,new_abstract,Topic
0,5e0c586f047ff909c8ed3fe171c8975a90608d08,Neutralizing antibodies against porcine epidem...,"Porcine epidemic diarrhea virus (PEDV), which ...",BACKGROUND: Porcine epidemic diarrhea virus (P...,4
1,1579fbff7af9b156c6f49fee0526e48f852ea460,A Recombinant Newcastle Disease Virus (NDV) Ex...,"Generation of rNDVs expressing S1, S2 or S pro...",Infectious bronchitis virus (IBV) causes a hig...,4
3,38aa050ad79d8a1d7022c33535255ce9d47914e5,Potent Inhibition of Junín Virus Infection by ...,Arenaviruses are enveloped RNA viruses with bi...,The new world arenavirus Junín virus (JUNV) is...,3
4,61722c462b054f36461375e96e502cbf22648c04,2 convergent Research center for emerging Viru...,"In this study, the anti-dengue activity of nic...",Dengue fever is one of the most important mosq...,3
5,7107f088cbed45d8a06a026276ccf4d602d50f10,Microglia Play a Major Role in Direct Viral-In...,Microglia are specialized macrophages of the C...,Microglia are the resident macrophage-like pop...,4
